In [ ]:
import pickle
import torch 
import ast
from tqdm import tqdm
import json
from itertools import islice
from torch.utils.data import TensorDataset, DataLoader 

In [ ]:
from ResearchGraphDataset import * 
import CollaborationPredictorglm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:496: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:353: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/root/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` fro

In [ ]:
import pandas as pd
df = pd.read_json('df.json')
df = df[df['publication_year']>=2020]

with open('splits_New.pkl', 'rb') as f:
	splits = pickle.load(f)

with open("data.json", "r", encoding="utf-8") as f: # prompt_data_preprocess获取的节点数据
    data = json.load(f)
    
    
idx = pd.read_csv(r'pos_idx.csv')

/root/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:353: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Processing papers: 100%|██████████| 12209/12209 [00:01<00:00, 6106.98it/s]
Processing collaborations: 12209it [00:01, 11508.17it/s]


In [ ]:
dataset = ResearchGraphDataset(df,splits, max_authors = 2) 
dataset.splits = splits
predictor = CollaborationPredictorglm.CollaborationPredictorglm(dataset,data) 

# 构建样本

# 

In [ ]:
split=2023
splits = dataset.splits[split]
train_g = dataset._add_topological_features(splits['train'])
sample_dataset = dataset.generate_candidate_pool(train_g)
train_loader = DataLoader(sample_dataset, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
best_samples = dataset.Sample_Optimization(train_loader,train_g)

# 预测

In [14]:
active_authors,author_id_to_idx,idx_to_author_id = dataset.generate_authorid2idx(2023)
all_pairs = list(data["dist"].keys())

In [ ]:
llm_cache = {}
error_list = []
for idx in tqdm(idx['idx']):
    key = int(idx)
    pairs = [eval(k) for k in all_pairs if key in eval(k)]
    for p in pairs:
        src_nodes = key
        try:
            dst_nodes = [n for n in p if n!= key][0]
        except:
            continue
        try:
            result = predictor.predict((src_nodes, dst_nodes), graph=train_g)
            llm_cache[(src_nodes, dst_nodes)] = result
        except:
            error_list.append((src_nodes, dst_nodes))
            continue 

  0%|          | 0/458 [00:00<?, ?it/s]

发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}


 42%|████▏     | 194/458 [2:01:07<2:26:27, 33.29s/it]

发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}
发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}
发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}


 43%|████▎     | 195/458 [2:01:11<1:46:39, 24.33s/it]

...

 90%|█████████ | 414/458 [4:03:43<25:40, 35.01s/it]  

发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}


 95%|█████████▌| 436/458 [4:18:53<18:03, 49.25s/it]

发生异常: Error code: 400, with error text {"contentFilter":[{"level":1,"role":"user"}],"error":{"code":"1301","message":"系统检测到输入或生成内容可能包含不安全或敏感内容，请您避免输入易产生敏感内容的提示语，感谢您的配合。"}}


100%|██████████| 458/458 [4:30:49<00:00, 35.48s/it]


In [ ]:
new_llm_cache = {}
for row_key, cols in llm_cache.items():
    row_str = str(row_key)
    for col_key, value in cols.items():
        new_llm_cache.setdefault(col_key, {})[row_str] = value
with open(f'main.json', "w") as f:
    json.dump(new_llm_cache, f)
print('==='+'saved'+'===')

===2023 saved===
